In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/kenn/anaconda3/envs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 思路：把一行像素作为一个x，把每一行作为一个输入节点，用RNN分类图片
# hyperparameters
lr = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

n_inputs = 28 # 输入的一行的长度（每个输入节点的维度）
n_steps = 28 # RNN的长度（输入节点的个数）
n_hidden_units = 128
n_classes = 10

In [4]:
# 输入层
x = tf.placeholder(tf.float32, [None,n_steps,n_inputs])
y = tf.placeholder(tf.float32, [None,n_classes])

In [5]:
# 权重
weights = {
     #第一层：28x128
    "in":tf.Variable(tf.random_normal([n_inputs, n_hidden_units]),dtype=tf.float32),
    #输出层：128x10
    "out":tf.Variable(tf.random_normal([n_hidden_units, n_classes]), dtype = tf.float32)
}

biases = {
    "in":tf.Variable(tf.constant(0.1,shape=[n_hidden_units]), dtype=tf.float32),
    "out":tf.Variable(tf.constant(0.1,shape=[n_classes]), dtype = tf.float32)
}

In [6]:
def RNN(X, weights, biases):
    # input ###
    
    # X.shape = [128 batch, 28 steps, 28 inputs]
    X = tf.reshape(X, [-1, n_inputs]) # 把整个batch的128个图片拆分成128x28行，28列。也就是一个X只有1x28
    X_in = tf.matmul(X, weights["in"]) + biases["in"] # 计算后把每一行像素扩展为128个隐含单元，得到128x28行，128列
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units]) # reshape回来，变成128个样本，每一个样本是28x128的矩阵
    
    # cell ###
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    # 初始forget_bias为1，不忘却以前的东西
    # state_is_tuple=True 表示以tuple形式生成(c_state,m_state)
    _init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)
    #time_major=False 时间维度是不是第一个维度，本例中维度定义为batch，step(time),inputs，第二个维度才是时间维度
    #states就是最后一个节点的输出
    
    # final result###
    results = tf.matmul(states[1], weights["out"]) + biases["out"]

    return results

In [7]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        #x_image = tf.reshape(xs,[-1,28,28]) 也可以在placeholder下定义reshape
        sess.run(train_op, feed_dict = {x:batch_xs, y:batch_ys})
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict = {x:batch_xs, y:batch_ys}))
        step+=1

0.1015625
0.6875
0.78125
0.8359375
0.8515625
0.8515625
0.890625
0.8515625
0.8984375
0.9140625
0.9140625
0.8515625
0.9296875
0.921875
0.9296875
0.9296875
0.96875
0.9375
0.96875
0.9453125
0.953125
0.984375
0.953125
0.9609375
0.953125
0.953125
0.96875
0.96875
0.9765625
0.984375
0.9765625
0.96875
0.96875
0.96875
0.9765625
0.984375
0.9609375
0.9609375
0.984375
0.9765625
